# Character-Level LSTM in PyTorch

In this notebook, we'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, we will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

First let's load in our required resources for data loading and model creation.

In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

---
## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use.

In [0]:
# open text file and read in the data as `text`
with open("anna.txt", 'r') as f:
    text = f.read()

In [5]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, we're creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [6]:
# encode the text and map each character to an integer and vice versa
# we create two dictionaries
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])
len(encoded)

1985223

In [49]:
!nvidia-smi

Thu Mar 12 04:05:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    43W / 250W |   1393MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

And we can see those same characters from above, encoded as integers.

In [50]:
encoded[:100]

array([44, 26, 35, 41, 15, 37, 71, 22, 29, 56, 56, 56, 62, 35, 41, 41,  6,
       22, 14, 35, 77, 72, 34, 72, 37, 53, 22, 35, 71, 37, 22, 35, 34, 34,
       22, 35, 34, 72, 17, 37,  5, 22, 37, 30, 37, 71,  6, 22, 66, 67, 26,
       35, 41, 41,  6, 22, 14, 35, 77, 72, 34,  6, 22, 72, 53, 22, 66, 67,
       26, 35, 41, 41,  6, 22, 72, 67, 22, 72, 15, 53, 22, 10,  2, 67, 56,
        2, 35,  6, 64, 56, 56, 70, 30, 37, 71,  6, 15, 26, 72, 67])

---
## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [0]:
def one_hot_encode(arr, n_labels):
    # initialize the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # fill the appropriate elements with one
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1

    # finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [52]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


---
## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches.**

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches.** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches.**

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [0]:
def get_batches(arr, batch_size, seq_length):
    """
    Create a generator that returns batches of size 
    `batch_size x seq_length` from `arr`.

    Arguments:
    ==========
    :param arr: Array we want to make batches from
    :param batch_size: Batch size, number of sequences per batch
    :param seq_length: Number of encoded chars in each sequence
    """

    ### TODO: Get the number of batches we make
    n_batches = len(arr) // (batch_size * seq_length)

    ### TODO: Keep only enough characters to make full batches
    arr = arr[:(n_batches*batch_size*seq_length)]

    ### TODO: Reshape into batch_size rows
    arr = arr.reshape(batch_size, -1)

    ### TODO:iterate over the batches using a sliding window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # the features
        x = arr[:, n:n+seq_length]
        # the targets, shifted by one
        y = np.zeros_like(x)
        try:        # takes care of the case when n+seq_length is the last element
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

---
### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [0]:
batches = get_batches(encoded, 8, 50)
x,y = next(batches)

In [55]:
print('x\n', x[:10, :10])
print('y\n', y[:10, :10])

x
 [[44 26 35 41 15 37 71 22 29 56]
 [53 10 67 22 15 26 35 15 22 35]
 [37 67 61 22 10 71 22 35 22 14]
 [53 22 15 26 37 22 11 26 72 37]
 [22 53 35  2 22 26 37 71 22 15]
 [11 66 53 53 72 10 67 22 35 67]
 [22 51 67 67 35 22 26 35 61 22]
 [18 68 34 10 67 53 17  6 64 22]]
y
 [[26 35 41 15 37 71 22 29 56 56]
 [10 67 22 15 26 35 15 22 35 15]
 [67 61 22 10 71 22 35 22 14 10]
 [22 15 26 37 22 11 26 72 37 14]
 [53 35  2 22 26 37 71 22 15 37]
 [66 53 53 72 10 67 22 35 67 61]
 [51 67 67 35 22 26 35 61 22 53]
 [68 34 10 67 53 17  6 64 22  1]]


---
## Defining the network with PyTorch

Below is where we'll define the network.

Next, we'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. We've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, we pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [56]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout=drop_prob, 
                            batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.chars))
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
#         print(out.shape)
        out = self.fc(out)
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.contiguous().view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [59]:
## TODO: set you model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [60]:
batch_size = 128
seq_length = 100
n_epochs =  50# start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, 
      lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 3.2692... Val Loss: 3.2009
Epoch: 1/50... Step: 20... Loss: 3.1521... Val Loss: 3.1358
Epoch: 1/50... Step: 30... Loss: 3.1461... Val Loss: 3.1242
Epoch: 1/50... Step: 40... Loss: 3.1162... Val Loss: 3.1198
Epoch: 1/50... Step: 50... Loss: 3.1447... Val Loss: 3.1178
Epoch: 1/50... Step: 60... Loss: 3.1174... Val Loss: 3.1153
Epoch: 1/50... Step: 70... Loss: 3.1061... Val Loss: 3.1123
Epoch: 1/50... Step: 80... Loss: 3.1188... Val Loss: 3.1054
Epoch: 1/50... Step: 90... Loss: 3.1074... Val Loss: 3.0905
Epoch: 1/50... Step: 100... Loss: 3.0656... Val Loss: 3.0516
Epoch: 1/50... Step: 110... Loss: 3.0249... Val Loss: 2.9994
Epoch: 1/50... Step: 120... Loss: 2.9482... Val Loss: 2.8931
Epoch: 1/50... Step: 130... Loss: 2.8251... Val Loss: 2.7783
Epoch: 2/50... Step: 140... Loss: 2.7223... Val Loss: 2.6606
Epoch: 2/50... Step: 150... Loss: 2.6267... Val Loss: 2.5690
Epoch: 2/50... Step: 160... Loss: 2.5610... Val Loss: 2.5160
Epoch: 2/50... Step: 170... Loss: